In [4]:
%pip install plotly seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Cloud Security Analysis with Demo Data
This notebook analyzes security findings from our demo data generators for:
1. IAM Entitlements
2. Storage Security

We'll use this analysis to understand potential security risks and remediation steps without needing actual cloud credentials.

In [5]:
# Import required libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
import plotly.express as px
import plotly.graph_objects as go

# Set up plotting style
sns.set_theme()
%matplotlib inline

## 1. Load Demo Data
Let's load both the IAM entitlement and storage security findings.

In [7]:
# Load IAM entitlement findings
with open('c:/Users/HP/OneDrive/Documents/CloudGuardStack/entitlement_report.json', 'r') as f:
    iam_report = json.load(f)

# Load storage security findings
with open('c:/Users/HP/OneDrive/Documents/CloudGuardStack/storage_demo_findings.json', 'r') as f:
    storage_report = json.load(f)

# Display scan metadata
display(Markdown("### IAM Scan Overview"))
display(Markdown(f"- **Scan Time**: {iam_report['scan_metadata']['scan_time']}"))
display(Markdown(f"- **Account ID**: {iam_report['scan_metadata']['account_id']}"))
display(Markdown(f"- **Total Findings**: {iam_report['scan_metadata']['total_findings']}"))

display(Markdown("\n### Storage Scan Overview"))
display(Markdown(f"- **Scan Time**: {storage_report['scan_metadata']['scan_time']}"))
display(Markdown(f"- **Environment**: {storage_report['scan_metadata']['environment']}"))
display(Markdown(f"- **Total Public Buckets**: {storage_report['summary']['total_public_buckets']}"))
display(Markdown(f"- **Total Sensitive Findings**: {storage_report['summary']['total_sensitive_findings']}"))

### IAM Scan Overview

- **Scan Time**: 2025-11-02T14:35:47.192670

- **Account ID**: 123456789012

- **Total Findings**: 12


### Storage Scan Overview

- **Scan Time**: 2025-11-02T14:34:28.512261

- **Environment**: demo

- **Total Public Buckets**: 3

- **Total Sensitive Findings**: 3

## 2. IAM Risk Analysis
Let's analyze the IAM security findings and risk scores.

In [8]:
# Create risk score DataFrame
risk_df = pd.DataFrame.from_dict(iam_report['risk_scores'], orient='index', columns=['risk_score'])
risk_df = risk_df.sort_values('risk_score', ascending=False)

print("Top High-Risk IAM Entities:")
display(risk_df.head())

# Plot risk distribution
fig = px.bar(
    risk_df.reset_index(),
    x='index',
    y='risk_score',
    title='IAM Entity Risk Scores',
    labels={'index': 'Entity', 'risk_score': 'Risk Score'},
    color='risk_score',
    color_continuous_scale='reds'
)
fig.show()

# Analyze findings by type
findings_data = []
for finding_type, items in iam_report['findings'].items():
    for item in items:
        findings_data.append({
            'finding_type': finding_type,
            'entity_name': item.get('entity_name', 'unknown'),
            'risk_level': item.get('risk_level', 'MEDIUM')
        })

findings_df = pd.DataFrame(findings_data)

# Create pie chart of finding types
fig = px.pie(
    findings_df,
    names='finding_type',
    title='Distribution of IAM Finding Types'
)
fig.show()

Top High-Risk IAM Entities:


,risk_score
admin-user,95.0
CustomPolicy1,90.0
dev-user1,85.0
temp-admin,80.0
BackupOperator,70.0


## 3. Storage Security Analysis
Now let's analyze the storage security findings.

In [9]:
# Analyze public buckets
public_buckets_df = pd.DataFrame(storage_report['public_buckets'])
display(Markdown("### Public Bucket Analysis"))
display(public_buckets_df[['bucket_name', 'public_access', 'risk_level', 'business_impact']])

# Create visualization of public bucket risks
fig = px.bar(
    public_buckets_df,
    x='bucket_name',
    y='size_gb',
    color='risk_level',
    title='Public Bucket Sizes and Risk Levels',
    labels={'size_gb': 'Size (GB)', 'bucket_name': 'Bucket Name'},
    color_discrete_map={
        'CRITICAL': '#ff0d0d',
        'HIGH': '#ff4e11',
        'MEDIUM': '#ff8e15'
    }
)
fig.show()

# Analyze sensitive data findings
sensitive_df = pd.DataFrame(storage_report['sensitive_data_findings'])
display(Markdown("\n### Sensitive Data Findings"))
display(sensitive_df[['bucket_name', 'object_key', 'risk_level', 'business_impact']])

# Create visualization of configuration issues
config_df = pd.DataFrame(storage_report['insecure_configurations'])
fig = px.pie(
    config_df,
    names='issue',
    title='Distribution of Security Configuration Issues'
)
fig.show()

### Public Bucket Analysis

,bucket_name,public_access,risk_level,business_impact
0,web-assets-prod,Bucket Policy - Public Read,CRITICAL,Customer-facing web assets exposed
1,user-uploads,Bucket ACL - Public Write,HIGH,Potential for malicious file uploads
2,NaN,Container - Blob Public Access,HIGH,Backup data potentially exposed



### Sensitive Data Findings

,bucket_name,object_key,risk_level,business_impact
0,config-files,production/config.json,CRITICAL,Production AWS credentials exposed
1,backup-data,ssh-keys/private-key.pem,HIGH,SSH private key exposed
2,temp-storage,database-backup/secrets.env,HIGH,Database credentials and connection strings ex...


## 4. Remediation Planning
Let's analyze the recommended remediation steps for both IAM and storage findings.

In [10]:
# Create remediation plan DataFrame
iam_remediation_df = pd.DataFrame(iam_report['remediation_suggestions'])
display(Markdown("### IAM Remediation Priorities"))
display(iam_remediation_df[['entity', 'priority', 'suggestion']])

# Display storage remediation case studies
display(Markdown("\n### Storage Security Case Studies"))
for case in storage_report['case_studies']:
    display(Markdown(f"#### {case['title']}"))
    display(Markdown(f"**Scenario**: {case['scenario']}"))
    display(Markdown("\n**Findings:**"))
    for finding in case['findings']:
        display(Markdown(f"- {finding}"))
    display(Markdown("\n**Remediation Steps:**"))
    for step in case['remediation_steps']:
        display(Markdown(f"- {step}"))
    display(Markdown("\n---"))

### IAM Remediation Priorities

,entity,priority,suggestion
0,admin-user,CRITICAL,Implement least privilege access model
1,CustomPolicy1,CRITICAL,Replace custom policy with restricted permissions
2,dev-user1,HIGH,Restrict development environment access



### Storage Security Case Studies

#### Public Web Assets Exposure

**Scenario**: Marketing team deployed web assets to S3 without proper security review

...

- Implemented unified storage security policy

- Created cloud-agnostic security templates

- Set up cross-cloud security monitoring

- Standardized encryption and access controls


---